<a href="https://colab.research.google.com/github/ido177/colab/blob/main/subrequest/subrequest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
6.33MB [00:00, 99.7MB/s]


# Загрузка БД

In [3]:
import pandas as pd
import numpy as np
import sqlite3

In [4]:
sqlite3.sqlite_version

'3.36.0'

In [5]:
con = sqlite3.connect('db')

In [6]:
!gdown --id 1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR

Downloading...
From: https://drive.google.com/uc?id=1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 25.0MB/s]


In [7]:
df = pd.read_csv('/content/german_credit_augmented.csv')

In [8]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


head выводит 5 первых значений, tail - 5 последних


In [9]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'],format='%Y-%m-%d %H:%M:%S')

to_datetime меняет формат данных в дату
% помогает указать переменную

In [10]:
df.dtypes

age                          int64
sex                         object
job                          int64
housing                     object
saving_accounts             object
checking_account            object
credit_amount                int64
duration                     int64
purpose                     object
default                      int64
contract_dt         datetime64[ns]
client_id                    int64
dtype: object

возможные типы данных

In [11]:
df.to_sql('german_credit',con,index=False,if_exists='replace')

Заливаем таблицу в БД
Название, выбор бд, наличие индекса, замена существующих

In [12]:
def select(sql):
  return pd.read_sql(sql,con)

функция селекта

# Подзапросы

In [13]:
t = pd.DataFrame({'id':[1,1,2,2,3],
                   'name':['a','b','c','d','e']})

In [14]:
t.to_sql('dupl_task', con, index=False,if_exists='replace')

In [22]:
sub_sql = '''select t.id 
from dupl_task t
group by t.id
having count(*) > 1'''

having используется в group by

In [17]:
select(sql)

,id,count(*)
0,1,2
1,2,2


In [24]:
sql = f'''select *
from dupl_task t 
where t.id in (
{sub_sql}
)
'''

In [25]:
select(sql)

,id,name
0,1,a
1,1,b
2,2,c
3,2,d


In [ ]:
sql = ''' 
drop table if exists dupls
create table dupls as
select t.id from dupls_test t
group by t.id
having count(*) > 1
'''